In [2]:
# TODO:
# +1. Update bert 0.6.1 vs 0.4.0  See https://github.com/huggingface/pytorch-pretrained-BERT/issues/328
# 2. add crf on the top bert
# +3. calculate probability of a sequence
# 4. Try standard CRF with features from ACL paper
# +5. Try other dataset (CONLL-2003, OntoNotes 5.0, NLPBA (http://www.nactem.ac.uk/tsujii/GENIA/ERtask/report.html), BioCreative)
# +7. Draw plots for CAD, HYP, DIABETES
# +8. Try document-level annotations
# +9. Create Bi-LSTM CRF baseline using Flair
# 10. Разобраться с тем как оценивали участники результаты на проверочном корпусе What is evaluated in i2b2 ? 

# 11. Take small amount of unannotated data, build dictionary and try to outperform the dictionary baseline
# 12. Show that language models are super cool for active learning since you can use just a shallow architecture for training (faster)
# 13. Just show that bert kills old baselines.
# 14. Unlexicalized BERT for active learning (Poked bert delex.py)
# 15. Propose a self-attention network for tagging and show it is better than LSTM
# + 16. New variants of ranking procedure. Most uncertain among those that get positive labels. Most uncertain among tokens that have positive labels. 
# (yeah it seems to work at some point)

# Calculate % of data by using number of tokens.
# Use dictionary-based procedure to generate first annotated dataset
# + Token evaluation metrics
# + Try BERT without fine tuning Работает так себе ~45% f1
# + evaluate bert on test data
# Посмотреть, как работают скрипты оценки качества в i2b2
# Fix sentence splitting procedure <===================
# Fine-tune bert model and train only classifier https://github.com/huggingface/pytorch-pretrained-BERT/tree/master/examples/lm_finetuning
# + Fix performance issue: generate batch with the length of the biggest sentence in batch
# + Implement early stopping on dev set 
# + Rerun all experiments with bert on new test data 
# + Fix AL pipeline: retrain model from scratch, etc. 
# Widget: implement updating of text field with selection in Javascript. Then activate text field and   catch callback in python
# + Check how we can deal with gpu memory allocation problem
# + Make charts to plot several strategies at once 
# Inclusive, exclusive evaluation.
# Measure time till convergence 
# Improve speed of flair Trainer, make universal trainer

# Vadim incorrect validation set need to take part of train instead of test
# + Make conversion stage from i2b2 format into conll. Use conll2003 format as the primary for experiments
# Make a wrapper around Flair model
# Make functions for testing Flair models
# New tokenizers

In [ ]:
!pip show pytorch_transformers seqeval flair

In [ ]:
!pip install git+https://github.com/IINemo/libact.git@seq

In [ ]:
#!pip install -e ./packages/libact/
!pip uninstall -y actleto
!pip install git+https://github.com/IINemo/active_learning_toolbox.git@seq

In [ ]:
!pip install -e ../../activer_learning_toolbox

# Initialization

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import sys
sys.path += ['../../bert_sequence_tagger/src/']

In [2]:
import torch

device = torch.device('cuda')
for i in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_name(i))

Tesla V100-DGXS-16GB


In [3]:
from logger import initialize_logger
logger = initialize_logger('../workdir/logs/biomed_ie.log', 'biomed_ie')
#initialize_logger('../workdir/logs/sequence_tagger_bert.log', 'sequence_tagger_bert')

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import json

# Global parameters

In [5]:
CACHE_DIR = '../workdir/cache'

# MAX_LEN = 150
BATCH_SIZE = 32
#BATCH_SIZE = 16
MAX_LEN = 100
#BATCH_SIZE = 100
#BATCH_SIZE_PRED = 1200 # 1600
#BATCH_SIZE_PRED = 1200 # 1600
BATCH_SIZE_PRED = 1500
PRED_BATCH_SIZE = BATCH_SIZE_PRED
random_state = 2019

EARLY_STOPPING = 1
MAX_RETRAIN_EPOCHS = 30
#MAX_RETRAIN_EPOCHS = 15
MAX_N_EPOCHS = MAX_RETRAIN_EPOCHS

N_SAMPLES_PER_AL_ITER = 30
N_AL_PASSES = 5
N_AL_ITERATIONS = 25
# N_AL_PASSES = 1
# N_AL_ITERATIONS = 1

BASE_LR = 5e-5
LEARNING_RATE = BASE_LR

VALIDATION_RATIO = 0.25

MAX_TO_ANNEAL = 3
ANNEAL_FACTOR = 0.5
PATIENCE = 2
WEIGHT_DECAY = 0.01

SEED_POSITIVE = 10
SEED_RANDOM = 40

BIO_BERT = '../workdir/bio_bert/torch2/'
OUTPUT_FILE_PATH = '../workdir/experiments/17_biobert/'
#OUTPUT_FILE_PATH = '../workdir/experiments/debug'

# Load dataset

In [6]:
from flair.datasets import ColumnCorpus

corpora = {}

data_folder = '../workdir/i2b2/conll/'
for attr in ['hypertension', 'cad', 'diabetes']:
    corpora[attr] = ColumnCorpus(data_folder, {0 : 'text', 1 : 'ner'},
                                 train_file=f'i2b2_training_{attr}.conll',
                                 test_file=f'i2b2_testing_{attr}.conll',
                                 dev_file=f'i2b2_testing_{attr}.conll')

data_folder = '../workdir/genia/conll/'
corpora['genia'] = ColumnCorpus(data_folder, {0 : 'text', 1 : 'ner'},
                                train_file='Genia4ERtask1.iob2',
                                test_file='Genia4EReval1.iob2',
                                dev_file='Genia4EReval1.iob2')    

2019-09-29 21:09:43,953 Reading data from ../workdir/i2b2/conll
2019-09-29 21:09:43,953 Train: ../workdir/i2b2/conll/i2b2_training_hypertension.conll
2019-09-29 21:09:43,954 Dev: ../workdir/i2b2/conll/i2b2_testing_hypertension.conll
2019-09-29 21:09:43,954 Test: ../workdir/i2b2/conll/i2b2_testing_hypertension.conll
2019-09-29 21:09:49,425 Reading data from ../workdir/i2b2/conll
2019-09-29 21:09:49,426 Train: ../workdir/i2b2/conll/i2b2_training_cad.conll
2019-09-29 21:09:49,427 Dev: ../workdir/i2b2/conll/i2b2_testing_cad.conll
2019-09-29 21:09:49,428 Test: ../workdir/i2b2/conll/i2b2_testing_cad.conll
2019-09-29 21:10:02,100 Reading data from ../workdir/i2b2/conll
2019-09-29 21:10:02,101 Train: ../workdir/i2b2/conll/i2b2_training_diabetes.conll
2019-09-29 21:10:02,102 Dev: ../workdir/i2b2/conll/i2b2_testing_diabetes.conll
2019-09-29 21:10:02,103 Test: ../workdir/i2b2/conll/i2b2_testing_diabetes.conll
2019-09-29 21:10:08,374 Reading data from ../workdir/genia/conll
2019-09-29 21:10:08,375

# Run experiments

In [7]:
from bert_active_learning_exp import run_experiment_al
from active_learning_seq import LibActNN, LibActNNPositiveLessCertain, RandomSamplingWithRetraining
from libact.query_strategies import UncertaintySampling, RandomSampling

from bert_sequence_tagger import SequenceTaggerBert, BertForTokenClassificationCustom, ModelTrainerBert
from bert_sequence_tagger.bert_utils import get_parameters_without_decay

from pytorch_transformers import BertTokenizer, AdamW, WarmupLinearSchedule, WarmupConstantSchedule
from torch.optim.lr_scheduler import ReduceLROnPlateau

from bert_sequence_tagger.metrics import f1_entity_level, f1_token_level


BERT_TOKENIZER = BertTokenizer.from_pretrained('bert-base-cased', 
                                               cache_dir=CACHE_DIR, 
                                               do_lower_case=False)

def create_libact_adaptor(tag2index, index2tag, adaptor_type, *args):
    def model_ctor():
        model = BertForTokenClassificationCustom.from_pretrained(BIO_BERT,
                                                                 cache_dir=CACHE_DIR, 
                                                                 num_labels=len(tag2index)).cuda()

        seq_tagger = SequenceTaggerBert(model, BERT_TOKENIZER, idx2tag=index2tag, tag2idx=tag2index)
        
        return seq_tagger
    
    def trainer_ctor(seq_tagger, corpus_len, train_dataloader, val_dataloader):
        optimizer = AdamW(get_parameters_without_decay(seq_tagger._bert_model),
                          lr=LEARNING_RATE, betas=(0.9, 0.999), 
                          eps=1e-6, weight_decay=0.01, correct_bias=True)

        lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=ANNEAL_FACTOR, patience=PATIENCE)
        
        trainer = ModelTrainerBert(seq_tagger, 
                                   optimizer, 
                                   lr_scheduler,
                                   train_dataloader, 
                                   val_dataloader,
                                   update_scheduler='ee',
                                   keep_best_model=True,
                                   restore_bm_on_lr_change=True,
                                   max_grad_norm=1.,
                                   validation_metrics=[f1_entity_level],
                                   decision_metric=lambda metrics: metrics[0],
                                   smallest_lr=LEARNING_RATE / (MAX_TO_ANNEAL**(1./ANNEAL_FACTOR) + 0.1))
        
        return trainer
    
    return adaptor_type(*args, 
                        model_ctor=model_ctor,
                        trainer_ctor=trainer_ctor,
                        batch_size=BATCH_SIZE,
                        bs_pred=BATCH_SIZE_PRED,
                        train_from_scratch=True,
                        retrain_epochs=MAX_RETRAIN_EPOCHS,
                        valid_ratio=VALIDATION_RATIO)

In [8]:
# attr= 'diabetes'
# exp = {
#     'corpus' : corpora[attr],
#     'output_file_path' : OUTPUT_FILE_PATH,
#     'ranking_strategy' : lambda trn_ds, _libact_model: UncertaintySampling(trn_ds, model=_libact_model, method='lc'),
#     'libact_adaptor_ctor' : lambda tag2index, index2tag: create_libact_adaptor(tag2index, index2tag, LibActNN),
#     'name' : f'{attr}_MNLP',
#     'n_passes' : N_AL_PASSES,
#     'n_al_iterations' : N_AL_ITERATIONS,
#     'max_len' : MAX_LEN,
#     'max_samples_number' : N_SAMPLES_PER_AL_ITER,
#     'seed_elems_per_class' : 10,
#     'n_seeds_random' : 40
# }
# run_experiment_al(**exp)

In [9]:
def create_i2b2_experiment_pack(corpus, attr):
     return [
#          {
#         'corpus' : corpus,
#         'output_file_path' : OUTPUT_FILE_PATH,
#         'ranking_strategy' : lambda trn_ds, _libact_model: RandomSamplingWithRetraining(trn_ds, model=_libact_model, method='lc'),
#         'libact_adaptor_ctor' : lambda tag2index, index2tag: create_libact_adaptor(tag2index, index2tag, LibActNN),
#         'name' : f'{attr}_i.i.d.',
#         'n_passes' : N_AL_PASSES,
#         'n_al_iterations' : N_AL_ITERATIONS,
#         'max_len' : MAX_LEN,
#         'max_samples_number' : N_SAMPLES_PER_AL_ITER,
#         'seed_elems_per_class' : SEED_POSITIVE,
#         'n_seeds_random' : SEED_RANDOM
#     },
    {
        'corpus' : corpus,
        'output_file_path' : OUTPUT_FILE_PATH,
        'ranking_strategy' : lambda trn_ds, _libact_model: UncertaintySampling(trn_ds, model=_libact_model, method='lc'),
        'libact_adaptor_ctor' : lambda tag2index, index2tag: create_libact_adaptor(tag2index, index2tag, LibActNN),
        'name' : f'{attr}_MNLP',
        'n_passes' : N_AL_PASSES,
        'n_al_iterations' : N_AL_ITERATIONS,
        'max_len' : MAX_LEN,
        'max_samples_number' : N_SAMPLES_PER_AL_ITER,
        'seed_elems_per_class' : SEED_POSITIVE,
        'n_seeds_random' : SEED_RANDOM
    },
    {
        'corpus' : corpus,
        'output_file_path' : OUTPUT_FILE_PATH,
        'ranking_strategy' : lambda trn_ds, _libact_model: UncertaintySampling(trn_ds, model=_libact_model, method='lc'),
        'libact_adaptor_ctor' : lambda tag2index, index2tag: create_libact_adaptor(tag2index, 
                                                                                   index2tag, 
                                                                                   LibActNNPositiveLessCertain, 
                                                                                   [e for e in tag2index.keys() if e.startswith('B')],
                                                                                   0.5), # reduction_factor
        'name' : f'{attr}_MNLP-mod',
        'n_passes' : N_AL_PASSES,
        'n_al_iterations' : N_AL_ITERATIONS,
        'max_len' : MAX_LEN,
        'max_samples_number' : N_SAMPLES_PER_AL_ITER,
        'seed_elems_per_class' : SEED_POSITIVE,
        'n_seeds_random' : SEED_RANDOM
    }
]

experiments = []
for attr, corpus in corpora.items():
    experiments += create_i2b2_experiment_pack(corpus, attr)

experiments

[{'corpus': <flair.datasets.ColumnCorpus at 0x7fe2dd8827b8>,
  'output_file_path': '../workdir/experiments/17_biobert/',
  'ranking_strategy': <function __main__.create_i2b2_experiment_pack.<locals>.<lambda>(trn_ds, _libact_model)>,
  'libact_adaptor_ctor': <function __main__.create_i2b2_experiment_pack.<locals>.<lambda>(tag2index, index2tag)>,
  'name': 'hypertension_MNLP',
  'n_passes': 5,
  'n_al_iterations': 25,
  'max_len': 100,
  'max_samples_number': 30,
  'seed_elems_per_class': 10,
  'n_seeds_random': 40},
 {'corpus': <flair.datasets.ColumnCorpus at 0x7fe2dd8827b8>,
  'output_file_path': '../workdir/experiments/17_biobert/',
  'ranking_strategy': <function __main__.create_i2b2_experiment_pack.<locals>.<lambda>(trn_ds, _libact_model)>,
  'libact_adaptor_ctor': <function __main__.create_i2b2_experiment_pack.<locals>.<lambda>(tag2index, index2tag)>,
  'name': 'hypertension_MNLP-mod',
  'n_passes': 5,
  'n_al_iterations': 25,
  'max_len': 100,
  'max_samples_number': 30,
  'seed_e

In [ ]:
for exp in experiments[2:4]:
    logger.info('#################### Experiment ######################')
    logger.info(exp)
    
    run_experiment_al(**exp)
    
    logger.info('################### Experiment finished ##############')

2019-09-29 21:10:46,826 - biomed_ie - INFO - #################### Experiment ######################
2019-09-29 21:10:46,826 #################### Experiment ######################
2019-09-29 21:10:46,828 - biomed_ie - INFO - {'corpus': <flair.datasets.ColumnCorpus object at 0x7fe2671dcc18>, 'output_file_path': '../workdir/experiments/17_biobert/', 'ranking_strategy': <function create_i2b2_experiment_pack.<locals>.<lambda> at 0x7fe2de74fbf8>, 'libact_adaptor_ctor': <function create_i2b2_experiment_pack.<locals>.<lambda> at 0x7fe3c8679c80>, 'name': 'cad_MNLP', 'n_passes': 5, 'n_al_iterations': 25, 'max_len': 100, 'max_samples_number': 30, 'seed_elems_per_class': 10, 'n_seeds_random': 40}
2019-09-29 21:10:46,828 {'corpus': <flair.datasets.ColumnCorpus object at 0x7fe2671dcc18>, 'output_file_path': '../workdir/experiments/17_biobert/', 'ranking_strategy': <function create_i2b2_experiment_pack.<locals>.<lambda> at 0x7fe2de74fbf8>, 'libact_adaptor_ctor': <function create_i2b2_experiment_pack.

Epoch:  53%|█████▎    | 16/30 [00:08<00:07,  1.83it/s]


2019-09-29 21:12:06,974 - biomed_ie - INFO - Performance on seed examples: {'f1_entity_level': 0}
2019-09-29 21:12:06,974 Performance on seed examples: {'f1_entity_level': 0}
2019-09-29 21:12:06,976 - biomed_ie - INFO - Active learning iteration: #0
2019-09-29 21:12:06,976 Active learning iteration: #0


Epoch:  67%|██████▋   | 20/30 [00:11<00:05,  1.79it/s]


2019-09-29 21:14:48,243 - biomed_ie - INFO - Performance: {'f1_entity_level': 0.0344}
2019-09-29 21:14:48,243 Performance: {'f1_entity_level': 0.0344}
2019-09-29 21:14:48,244 - biomed_ie - INFO - Active learning iteration: #1
2019-09-29 21:14:48,244 Active learning iteration: #1


...

Epoch:  57%|█████▋    | 17/30 [00:11<00:08,  1.53it/s]


2019-09-29 22:39:16,160 - biomed_ie - INFO - Performance: {'f1_entity_level': 0.00340522133938706}
2019-09-29 22:39:16,160 Performance: {'f1_entity_level': 0.00340522133938706}
2019-09-29 22:39:16,161 - biomed_ie - INFO - Active learning iteration: #1
2019-09-29 22:39:16,161 Active learning iteration: #1


Epoch:  67%|██████▋   | 20/30 [00:19<00:09,  1.03it/s]


2019-09-29 22:42:06,062 - biomed_ie - INFO - Performance: {'f1_entity_level': 0.09652173913043478}
2019-09-29 22:42:06,062 Performance: {'f1_entity_level': 0.09652173913043478}
2019-09-29 22:42:06,063 - biomed_ie - INFO - Active learning iteration: #2
2019-09-29 22:42:06,063 Active learning iteration: #2


Epoch:  57%|█████▋    | 17/30 [00:19<00:14,  1.12s/it]


# Analyze results

In [17]:
from exp_utils import print_al_stats

In [18]:
data_path = '../workdir/experiments_draw/diabetes_MNLP-mod.npy'
entire_dat_score = 0.718
print_al_stats(data_path, entire_dat_score)

(10,)
Al score:  0.6513872668796481
Entire data score: 0.718
Score ratio: 0.9072246056819612


In [19]:
data_path = '../workdir/experiments_draw/cad_MNLP-mod.npy'
entire_dat_score = 0.401
print_al_stats(data_path, entire_dat_score)

(10,)
Al score:  0.2841254596866301
Entire data score: 0.401
Score ratio: 0.708542293482868


In [20]:
data_path = '../workdir/experiments_draw/hypertension_MNLP-mod.npy'
entire_dat_score = 0.739
print_al_stats(data_path, entire_dat_score)

(10,)
Al score:  0.6053099221998488
Entire data score: 0.739
Score ratio: 0.8190932641405261


In [ ]:
import numpy as np

data_path = '../workdir/experiments_draw/diabetes_MNLP-mod.npy'
stats = np.load(data_path)

stats.mean()